#  Inflation Rate Dashboard 

## Overview
This code creates an interactive dashboard using Dash, a Python web application framework. The dashboard is designed to visualize historical U.S. inflation rates and predict future inflation rates based on a provided dataset.


In [1]:
pip install dash pandas plotly


Note: you may need to restart the kernel to use updated packages.


## Imports
The following libraries are imported at the beginning of the code:
- `dash` and related components for building the dashboard.
- `pandas` for data manipulation and analysis.
- `plotly.express` for creating interactive visualizations.


In [39]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px


## Dataset
The dataset used in this code contains historical U.S. inflation rate data. It should be in CSV format and include at least two columns: 'date' and 'value'. The 'date' column should contain dates, and the 'value' column should contain corresponding inflation rate values.
dataset link:https://www.kaggle.com/datasets/pavankrishnanarne/us-inflation-dataset-1947-present
## Loading Data
The dataset is loaded into a Pandas DataFrame using the `pd.read_csv()` function. The 'date' column is converted to a datetime format using `pd.to_datetime()` for consistent date handling.

In [40]:
# df = pd.read_csv("C:\Users\sroha\OneDrive\Desktop\US_inflation_rates.csv")
df = pd.read_csv("C:/Users/sroha/OneDrive/Desktop/US_inflation_rates.csv")
df['date'] = pd.to_datetime(df['date'])

In [41]:
df.head()

,date,value
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


## Initializing the Dash App
An instance of the Dash application is created using `dash.Dash(__name__)`.

In [42]:
app = dash.Dash(__name__)


## Layout
The dashboard layout is defined using HTML and Dash components:
- An H1 header displaying the title of the dashboard.
- A dropdown component (`dcc.Dropdown`) for selecting a specific date.
- A graph component (`dcc.Graph`) for displaying the inflation rates.
- A placeholder (`html.Div`) for displaying future inflation rate predictions.


In [43]:
# Define the layout
app.layout = html.Div([
    html.H1("U.S. Inflation Rate Dashboard"),
    
    # Dropdown for date selection
    dcc.Dropdown(
        id='dropdown-date',
        options=[
            {'label': date, 'value': date} for date in df['date']
        ],
        value=df['date'].min(),
        multi=False,
        placeholder="Select a Date"
    ),
    
    # Graph for displaying inflation rates
    dcc.Graph(id='inflation-graph'),
    
    # Placeholder for future predictions
    html.Div(id='future-predictions', style={'margin-top': '20px'})
])

## Callbacks
Two callback functions are defined:
1. `update_graph`: This function updates the graph based on the selected date from the dropdown. It filters the dataset to show inflation rates for the selected year using `plotly.express` to create an interactive line chart.
2. `update_future_predictions`: This function predicts the inflation rate for the year following the selected date. It calculates the moving average of the selected date's inflation rate and predicts a 5% increase for the next year.


In [44]:
# Callback function to update the graph
@app.callback(
    Output('inflation-graph', 'figure'),
    [Input('dropdown-date', 'value')]
)
def update_graph(selected_date):
    # Convert the selected_date to a datetime object
    
    selected_date = pd.to_datetime(selected_date)
    
    # Filter data based on selected date
    filtered_df = df[df['date'] == selected_date]
    
    # Create a Plotly figure
    fig = px.line(filtered_df, x='date', y='value', title=f'Inflation Rates for {selected_date.year}')
    
    return fig




In [45]:
# Callback function to update future year predictions
@app.callback(
    Output('future-predictions', 'children'),
    [Input('dropdown-date', 'value')]
)
def update_future_predictions(selected_date):
    selected_date = pd.to_datetime(selected_date)
    
    future_date = selected_date + pd.DateOffset(years=1)
    
    # Calculate moving average for the selected date
    selected_date_data = df[df['date'] == selected_date]
    moving_avg = selected_date_data['value'].mean()
    
    # Predict future inflation using moving average
    future_prediction = moving_avg * 1.05  # Assuming a 5% increase
    
    return f"Predicted inflation rate for {future_date.year}: {future_prediction:.2f}%"

## Running the App
The Dash app is run using the `app.run_server(debug=True)` command. The `if __name__ == '__main__':` block ensures that the app is only run when the script is executed directly.
Open your web browser and go to http://127.0.0.1:8050/ to interact with the dashboard.


In [46]:
if __name__ == '__main__':
    app.run_server(debug=True)